# Нейросетевые модели поиска. Часть III. Векторный поиск.

# Домашнее задание

Цель домашнего задания - научиться строить индексы для векторного поиска на БОЛЬШИХ объемах данных, добиваясь при этом оптимального качества и скорости. Работать будем с уже известным датасетом MS MARCO (точнее, с сэмплом из него). Сэмпл генерируется следующим кодом:

In [1]:
import datasets
from datasets import load_dataset
import pandas as pd
import numpy as np

In [2]:
msmarco_dataset = load_dataset("Tevatron/msmarco-passage")

Generating train split:   0%|          | 0/400782 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/6980 [00:00<?, ? examples/s]

Generating dl19 split:   0%|          | 0/43 [00:00<?, ? examples/s]

Generating dl20 split:   0%|          | 0/54 [00:00<?, ? examples/s]

In [3]:
rows = []
i = -1
j = -1
for row in msmarco_dataset['train']:
    i += 1
    qid = i
    query = row['query']
    for pos_ex in row['positive_passages']:
        j += 1
        docid = j
        title = pos_ex['title']
        text = pos_ex['text']
        mark = 1
        rows.append([qid, query, docid, title, text, mark])
    for neg_ex in row['negative_passages']:
        j += 1
        docid = j
        title = neg_ex['title']
        text = neg_ex['text']
        mark = 0
        rows.append([qid, query, docid, title, text, mark])
df = pd.DataFrame(rows, columns=['qid', 'query', 'docid', 'title', 'text', 'mark'])

In [4]:
df = df[:2000000]
df.to_csv('homework_sample.csv', index=False)

In [6]:
df

,qid,query,docid,title,text,mark
0,0,where is whitemarsh island,0,"Whitemarsh Island, Georgia","Whitemarsh Island, Georgia. Whitemarsh Island ...",1
1,0,where is whitemarsh island,1,What military strategy was used in the pacific?,the strategy of island hopping was used by the...,0
2,0,where is whitemarsh island,2,Whakaari / White Island,"For the island near Dunedin, see White Island,...",0
3,0,where is whitemarsh island,3,Jekyll Island,"Jekyll Island, at 5,700 acres, is the smallest...",0
4,0,where is whitemarsh island,4,Sibu Island,Sibu Island. A scuba diver at Sibu Island. Sib...,0
...,...,...,...,...,...,...
1999995,64923,richest man in mexico net worth,1999995,-,6. The next name on the list of richest actors...,0
1999996,64923,richest man in mexico net worth,1999996,-,[image via JJ Duncan] 1 Billy Ray Cyrus Net W...,0
1999997,64923,richest man in mexico net worth,1999997,-,"Jeff Bezos is the founder of Amazon, and one o...",0
1999998,64923,richest man in mexico net worth,1999998,-,Similar Celebrities: 1 Jeremy Mayfield Net Wo...,0


Баллы начисляются за выполнение следующих заданий:

# Предобработка данных [2 балла]

Возможно, прежде чем строить эмбеддинги из текстовых данных, стоит их предобработать? (использовать не только зону text, что-то сделать с пунктуацией, использовать зону не целиком). В случае использования только СЫРОЙ зоны text (как на семинаре) бал начислен не будет

# Подбор нейронки [1 балл]

На семинаре для варки эмбеддингов использовался Universal Sentence Encoder. Возможно, стоит попробовать что-то другое? (балл будет начислен в том случае, если для варки индексов из следующих пунктов будет использована/ы другая/ие нейронки - да, использовать разные нейронки для разных индексов можно)

# Метрика для оценки качества [1 балл]

Нужно реализовать метрику для оценки качества индексов - ndcg@10

# Варка и инференс индексов [до 6 баллов]

В рамках этой части задания нужно сварить индексы 3 типов - annoy, faiss, hnswlib. Параметры можно выбирать любые (стоит ориентироваться на качество индекса, требуемую память и перф (см. ниже).)

За 1 индекс можно получить:

    1 балл,  если индекс состоит из  300000 документов
    2 балла, если индекс состоит из 1000000 документов
    За индексы из 2000000 документов дополнительные баллы в этом пункте не начисляются, но только они могут участвовать в соревновании

Важно:

1. Эмбеддинги, которые кладутся в индекс, должны быть сгенерированы нейронкой и соответствовать документам из датасета, иначе баллы аннулируются (сложить 2 миллиона эмбеддингов вида [1,1,1,...,1] нельзя)

2. Чтобы получить баллы за индекс, необходимо не только сварить, но и проинференсить его: прогнать на нем набор запросов из датасета (~65k), оценить качество результата (используя метрику ndcg@10), оценить время инференса (напр., так как в семинаре). Без этого баллы начислены не будут

3. При варке и инференсе индекса нельзя использовать больше 16GB оперативной памяти (иначе начислится 0 баллов)

# Соревнование [до 10 баллов]

Проводится среди индексов размерами 2000000 документов (остальные не оцениваются) в двух дисциплинах: по качеству (величина ndcg@10) и скорости (насколько быстро удалось прогнать полный набор запросов на этом индексе)

Оценивается только один индекс (указывается сдающим домашку) - если вы сварили 3 индекса по 2 миллиона доков каждый, то укажите, какой участвует в соревновании (напишите в ноутбуке). Иначе выбор будет сделан рандомно из доступных вариантов

Разбалловка:


За первое место по качеству: 5 баллов
    
За второе место по качеству: 3 балла
    
За все остальные места по качеству, кроме последнего: 1 балл

За первое место по скорости (при не последнем месте по качеству): 5 баллов

За второе место по скорости (при не последнем месте по качеству): 3 балла

За все остальные места по скорости (при не последнем месте по качеству), кроме последнего: 1 балл

# Процедура сдачи

**Вам надо:**

- Форкнуть эту репу;
- Создать бранч, в котором вы дальше будете работать;
- Выполнить все или часть заданий ноутбука;
- Запушить ваш бранч и поставить Pull Request.

Проверяющий счекаутит вашу бранчу и проверит работу.